In [ ]:
!pip install -q -U transformers datasets bitsandbytes  trl peft  huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 20.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "meta-llama/Llama-3.2-1B-Instruct"

quantization_config_4bit = BitsAndBytesConfig(load_in_4bit=True)
quantization_config_8bit = BitsAndBytesConfig(load_in_8bit=True)

In [ ]:
from huggingface_hub import login
login()

In [ ]:
model_4bit = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config_4bit, device_map="auto")
model_8bit = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config_8bit, device_map="auto")

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
model_16bit = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model_32bit = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

In [ ]:
import torch

def print_model_size_in_mb_gb(model):
  param_size = 0
  for param in model.parameters():
    param_size += param.numel() * param.element_size()

  buffer_size = 0
  for buffer in model.buffers():
    buffer_size += buffer.numel() * buffer.element_size()

  size_bytes = param_size + buffer_size
  size_mb = size_bytes / (1024 * 1024)
  size_gb = size_mb / 1024

  print(f"Model size: {size_mb:.2f} MB ({size_gb:.2f} GB)")


In [ ]:
print_model_size_in_mb_gb(model_4bit)
print_model_size_in_mb_gb(model_8bit)
print_model_size_in_mb_gb(model_16bit)
print_model_size_in_mb_gb(model_32bit)

Model size: 965.13 MB (0.94 GB)
Model size: 1429.13 MB (1.40 GB)
Model size: 2357.13 MB (2.30 GB)
Model size: 4714.26 MB (4.60 GB)


In [ ]:
import torch

torch.set_printoptions(precision=8, sci_mode=False)

layer_quantized = model_32bit.model.layers[0]
weight_quantized = layer_quantized.self_attn.q_proj.weight

print("32 bit Weight value:")
print(weight_quantized)

32 bit Weight value:
Parameter containing:
tensor([[-0.01794434,  0.00662231,  0.02465820,  ..., -0.00872803,
         -0.01171875,  0.02014160],
        [ 0.01220703,  0.05932617,  0.05517578,  ..., -0.03320312,
         -0.01538086,  0.01080322],
        [ 0.01782227,  0.01550293,  0.03442383,  ..., -0.03857422,
         -0.03857422, -0.02758789],
        ...,
        [ 0.02978516,  0.03515625,  0.07128906,  ..., -0.07177734,
         -0.02648926, -0.02868652],
        [ 0.02258301, -0.02478027,  0.03515625,  ..., -0.01196289,
         -0.02868652, -0.01477051],
        [-0.02575684, -0.05371094, -0.01306152,  ...,  0.05419922,
          0.00958252, -0.00277710]], device='cuda:0', requires_grad=True)


In [ ]:
import torch

torch.set_printoptions(precision=8, sci_mode=False)

layer_quantized = model_8bit.model.layers[0]
weight_quantized = layer_quantized.self_attn.q_proj.weight

print("16 bit Weight value:")
print(weight_quantized)

16 bit Weight value:
Parameter containing:
Parameter(Int8Params([[-44,  16,  61,  ..., -22, -29,  50],
            [ 14,  70,  65,  ..., -39, -18,  13],
            [ 16,  14,  30,  ..., -34, -34, -24],
            ...,
            [ 17,  20,  40,  ..., -40, -15, -16],
            [ 32, -35,  50,  ..., -17, -41, -21],
            [-14, -30,  -7,  ...,  30,   5,  -2]], device='cuda:0',
           dtype=torch.int8))


In [ ]:
import torch

torch.set_printoptions(precision=8, sci_mode=False)

layer_quantized = model_4bit.model.layers[0]
weight_quantized = layer_quantized.self_attn.q_proj.weight

print("16 bit Weight value:")
print(weight_quantized)

16 bit Weight value:
Parameter containing:
Parameter(Params4bit([[214],
            [ 46],
            [203],
            ...,
            [228],
            [196],
            [ 25]], device='cuda:0', dtype=torch.uint8))


In [ ]:
layer_quantized = model_32bit.model.layers[0]
weight_quantized_max = layer_quantized.self_attn.q_proj.weight.max()
weight_quantized_min = layer_quantized.self_attn.q_proj.weight.min()

print("32 bit  Weight value:")
print(f"weight_quantized_max {weight_quantized_max}")
print(f"weight_quantized_min {weight_quantized_min}")

32 bit  Weight value:
weight_quantized_max 0.58203125
weight_quantized_min -0.67578125


In [ ]:
layer_quantized = model_8bit.model.layers[0]
weight_quantized_max = layer_quantized.self_attn.q_proj.weight.max()
weight_quantized_min = layer_quantized.self_attn.q_proj.weight.min()

print("8 bit  Weight value:")
print(f"weight_quantized_max {weight_quantized_max}")
print(f"weight_quantized_min {weight_quantized_min}")

8 bit  Weight value:
weight_quantized_max 127
weight_quantized_min -127


In [ ]:
model_32bit

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (ro

In [ ]:
model_16bit

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (ro

In [ ]:
model_8bit

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm)

In [ ]:
import torch

torch.cuda.empty_cache()
start_mem = torch.cuda.memory_allocated(device=0)
model_8bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config_8bit,
    device_map="auto"
)
end_mem = torch.cuda.memory_allocated(device=0)
used_mem = end_mem - start_mem
print(f"8-bit model GPU memory usage: {used_mem/1024**2:.2f} MB")

8-bit model GPU memory usage: 1431.57 MB
